In [1]:
%load_ext autoreload
%autoreload 2
import sys
import numpy as np
import pandas as pd
sys.path.append('putilities')
import utilibs as ul
import math
from sklearn.neighbors import KNeighborsClassifier

In [2]:
runways = pd.read_csv('dbase/runway.csv')
flights = pd.read_csv('dbase/flights.csv')
ms = pd.read_csv('dbase/measurements.csv')

In [4]:
flights.dep_aprt.unique()

array(['KGSO', 'KMSP', 'KCVG', 'KDTW', 'KPHL', 'KORF', 'KLEX', 'KRST',
       'KVPS', 'KDLH', 'KRAP', 'KFAR', 'KROC', 'KATL', 'KSTL', 'KMSN',
       'KDSM', 'KFNT', 'CYQR', 'KBHM', 'KOKC', 'KFSD', 'KPIT', 'CYUL',
       'KLSE', 'KCLE', nan, 'KCHS', 'KMEM', 'KSLC', 'KGFK', 'KGRR',
       'KHPN', 'KPWM', 'KMCI', 'KGPT', 'KOMA', 'KBTV', 'KEWR', 'KDFW',
       'KTYS', 'KCMH', 'KAZO', 'KXNA', 'KBNA', 'KBGR', 'KPNS', 'KGRB',
       'KCID', 'KORD', 'KMBS', 'KSGF', 'KPLN', 'KCLT', 'KSDF', 'KJAN',
       'KAUS', 'KJAX', 'KATW', 'KMOB', 'KSAV', 'KTVC', 'KASE', 'KICT',
       'KABE', 'KRDU', 'KTUL', 'LFFE', 'KMDT', 'KGJT', 'KDAY', 'KSHV',
       'KDCA', 'KARA', 'CYYC', 'KMLI', 'CYHZ', 'KLIT', 'KSBN', 'WPLN',
       'KMKE', 'KBUF', 'KAUG', 'KCMX', 'KLCQ', 'KGEO', 'CYXE', 'KFCM',
       'MMDA', 'CABO', 'KMZJ', 'DCBS', 'KHSV', 'KMSL', 'K6D9', 'KMIC',
       'GLH', 'CYHM', 'ALX_', 'KDTS', 'KMGJ', 'KSTP', 'KIDA'],
      dtype=object)

In [5]:
model = KNeighborsClassifier(n_neighbors=1)

In [9]:
#features = ['ms100', 'ms101']
#flights['dep_rwy'] = ''

features = ['ms102', 'ms103']
flights['arr_rwy'] = ''

In [10]:
disp = [500 * i for i in range(int(len(flights)/500))]
for idx, file in enumerate(flights.fname.tolist()):
    apt_icao = flights.dep_aprt.iloc[idx]
    rwy_data = runways[runways.airport_icao == apt_icao][['airport_icao', 'lat_deg', 'lon_deg', 'rwy_ident']]
    if rwy_data.empty:
        text = f"{idx:5.0f} ... {file}   -- empty airport"
    else:
        model.fit(rwy_data[['lat_deg', 'lon_deg']] * math.radians(1), rwy_data.rwy_ident)
        predict_data = ms[ms.fname == file][features]
        text = f"{idx:5.0f} ... {file} ... {apt_icao} ... {model.predict(predict_data)[0]}"
        flights.loc[flights.fname == file, 'arr_rwy'] = model.predict(predict_data)[0]
    if idx in disp:
        print(text)

    0 ... flight_10000.parquet ... KGSO ... 23R
  500 ... flight_10500.parquet ... KMSP ... 30L
 1000 ... flight_11000.parquet ... KPHL ... 09R
 1500 ... flight_11500.parquet ... KMSP ... 35
 2000 ... flight_12000.parquet ... KAZO ... 27
 2500 ... flight_12500.parquet ... KFNT ... 36
 3000 ... flight_13000.parquet ... KMSP ... 30R
 3500 ... flight_13500.parquet ... KRST ... 13
 4000 ... flight_14000.parquet ... KDTW ... 27R
 4500 ... flight_14500.parquet ... KHPN ... 11
 5000 ... flight_15000.parquet ... CYUL ... 10
 5500 ... flight_15500.parquet ... KEWR ... 11
 6000 ... flight_16000.parquet ... KSTL ... 30R
 6500 ... flight_16500.parquet ... KOKC ... 17L
 7000 ... flight_17000.parquet ... KMSP ... 35
 7500 ... flight_17500.parquet ... KDTW ... 22R
 8000 ... flight_18000.parquet ... KEWR ... 04L
 8500 ... flight_18500.parquet ... KMEM ... 36R
 9000 ... flight_19000.parquet ... KHPN ... 11
 9500 ... flight_19500.parquet ... KSLC ... 17
10000 ... flight_20000.parquet ... KMSP ... 30R
10

In [11]:
flights

,fname,orig_name,tail_id,dep_aprt,arr_aprt,fsize_mb,flag,addr,dep_rwy,arr_rwy
0,flight_10000.parquet,654200109211115,Tail_654_2,KGSO,KDTW,2.10,True,/storage/NASA-par/flight_10000.parquet,32,23R
1,flight_10001.parquet,654200110141815,Tail_654_2,KMSP,KHPN,2.40,True,/storage/NASA-par/flight_10001.parquet,30R,30R
2,flight_10002.parquet,654200108211001,Tail_654_2,KCVG,KMSP,2.35,True,/storage/NASA-par/flight_10002.parquet,27,18R
3,flight_10003.parquet,654200107241628,Tail_654_2,KDTW,KDSM,1.87,True,/storage/NASA-par/flight_10003.parquet,04R,04L
4,flight_10004.parquet,654200108311351,Tail_654_2,KPHL,KMEM,3.07,True,/storage/NASA-par/flight_10004.parquet,27L,09R
...,...,...,...,...,...,...,...,...,...,...
39525,flight_49525.parquet,655200302241443,Tail_655_7,KHPN,KMSP,3.11,True,/storage/NASA-par/flight_49525.parquet,34,11
39526,flight_49526.parquet,655200305161343,Tail_655_7,KMSP,KSDF,2.27,True,/storage/NASA-par/flight_49526.parquet,30R,30L
39527,flight_49527.parquet,655200307021004,Tail_655_7,KDTW,KDAY,1.13,True,/storage/NASA-par/flight_49527.parquet,21R,04L
39528,flight_49528.parquet,655200306280626,Tail_655_7,KBGR,KDTW,2.54,True,/storage/NASA-par/flight_49528.parquet,15,15


In [12]:
flights.to_csv("dbase/flights.csv")

In [1]:
532/60

8.866666666666667